# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:LoRa 
* Model: distilgpt2
* Evaluation approach: Trainer-based evaluation.
* Fine-tuning dataset: SST-2 (Stanford Sentiment Treebank)

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

**Some code was reused from the lab of the workpaces of the course.**

In [1]:
from datasets import load_dataset
import random
import torch
print(torch.cuda.is_available())  
dataset = load_dataset("glue", "sst2")
dataset
sample_size = 5000
random_indices = random.sample(range(len(dataset['train'])), sample_size)
dataset['train'] = dataset['train'].select(random_indices)
dataset


True


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

**Identify if the classes are imbalanced**

This training set is imbalanced but not extremly.

In [2]:
train_labels = dataset['train']['label']

negative_count = sum(label == 0 for label in train_labels)  # Label 0 represents negative sentiment
positive_count = sum(label == 1 for label in train_labels)  # Label 1 represents positive sentiment

print(f"Number of Negative Class samples (label 0): {negative_count}")
print(f"Number of Positive Class samples (label 1): {positive_count}")

Number of Negative Class samples (label 0): 2216
Number of Positive Class samples (label 1): 2784


In [3]:
from transformers import AutoTokenizer

def tokenize_function(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True)


tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = dataset.map(tokenize_function, batched=True)
tokenized_ds["train"] = tokenized_ds["train"].shuffle(seed=21)
tokenized_ds["validation"] = tokenized_ds["validation"].shuffle(seed=21)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
tokenized_ds["train"].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilgpt2",
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1},
)

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**How to try Imbalanced Classes**
To try imabalanced classes, I used this article 
https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/

For me, false negatives are particularly costly in this case. Why? If the model incorrectly classifies an entry as NEGATIVE when it's not, the company would likely implement retention strategies to try and change the customer's opinion, which would incur additional expenses. Given this context,I will calculate   F2 score whic emphasizes minimizing false negatives by weighing recall twice as much as precision.

In [7]:
!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 37.6 MB/s eta 0:00:00


In [13]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, recall_score, precision_score, f1_score,fbeta_score



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f2=fbeta_score(labels, predictions, beta=2)
    return {"f2-score": f2}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./polarity",
        per_device_eval_batch_size=16,  
        fp16=True,  
        logging_steps=100),
    eval_dataset=tokenized_ds['validation'],  
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

{'eval_loss': 6.920271396636963,
 'eval_f2-score': 0.8383685800604229,
 'eval_runtime': 21.5974,
 'eval_samples_per_second': 40.375,
 'eval_steps_per_second': 2.547}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

**Notice: I used the following reference to build  LORAConfig. https://github.com/huggingface/blog/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md**

In [19]:
from peft import LoraConfig, get_peft_model,TaskType
from transformers import AutoModelForSequenceClassification

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8, 
    lora_alpha=32,  
    bias='none',
    lora_dropout=0.1)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilgpt2",
    num_labels=2,
    id2label={0: "negative", 1: "positive"},
    label2id={"negative": 0, "positive": 1},
)

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

lora_model = get_peft_model(model, lora_config)
for name, param in lora_model.named_parameters():
    print(f"Parameter: {name}, Trainable: {param.requires_grad}")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Parameter: base_model.model.transformer.wte.weight, Trainable: False
Parameter: base_model.model.transformer.wpe.weight, Trainable: False
Parameter: base_model.model.transformer.h.0.ln_1.weight, Trainable: False
Parameter: base_model.model.transformer.h.0.ln_1.bias, Trainable: False
Parameter: base_model.model.transformer.h.0.attn.c_attn.weight, Trainable: False
Parameter: base_model.model.transformer.h.0.attn.c_attn.bias, Trainable: False
Parameter: base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight, Trainable: True
Parameter: base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight, Trainable: True
Parameter: base_model.model.transformer.h.0.attn.c_proj.weight, Trainable: False
Parameter: base_model.model.transformer.h.0.attn.c_proj.bias, Trainable: False
Parameter: base_model.model.transformer.h.0.ln_2.weight, Trainable: False
Parameter: base_model.model.transformer.h.0.ln_2.bias, Trainable: False
Parameter: base_model.model.transformer.h.0.mlp.c_fc.weight,

In [20]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import torch


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f2=fbeta_score(labels, predictions, beta=2)
    return {"f2-score": f2}

torch.cuda.empty_cache() 
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./polarity",
        learning_rate=2e-4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=3,
        weight_decay=0.01,
        logging_steps=100,
        fp16=False,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,F2-score
1,0.407800,0.419026,0.776902
2,0.391000,0.384073,0.864143
3,0.321500,0.372836,0.864695


Checkpoint destination directory ./polarity/checkpoint-625 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.37283584475517273,
 'eval_f2-score': 0.8646953405017921,
 'eval_runtime': 23.9148,
 'eval_samples_per_second': 36.463,
 'eval_steps_per_second': 4.558,
 'epoch': 3.0}

In [21]:
lora_model.save_pretrained("distilgpt2-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import PeftModel
from peft import AutoPeftModelForSequenceClassification
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f2=fbeta_score(labels, predictions, beta=2)
    return {"f2-score": f2}

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("distilgpt2-lora", num_labels=2)
lora_model.resize_token_embeddings(len(tokenizer))
lora_model.config.pad_token_id = tokenizer.pad_token_id
trainer_lora_model = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./polarity",
        per_device_eval_batch_size=8,  
        fp16=True,  
        logging_steps=100),
    eval_dataset=tokenized_ds['validation'],  
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
with torch.no_grad():  
    print(trainer_lora_model.evaluate())




Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.37283584475517273, 'eval_f2-score': 0.8646953405017921, 'eval_runtime': 23.2993, 'eval_samples_per_second': 37.426, 'eval_steps_per_second': 4.678}
